<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
%%capture
!pip install transformers
!pip install python-chess

In [178]:
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data_with_buckets.csv"
csv_path = f"{folder}/{csv_file}"


now = datetime.now()
dt_string = now.strftime("%m_%d_%Y_%H_%M")

model_file = "regression_model.pth"
datetime = dt_string
model_path = f"{folder}/{datetime}_{model_file}"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [179]:
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from imblearn.under_sampling import RandomUnderSampler

###Parameters

In [180]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    "batch_size": 1,
    "epochs": 5,
    "learning_rate": 0.00003,
    "warmup_steps": 0.01,
    "epsilon": 1e-8,
    "accum_iter": 8,
    "num_classes": 6,
    "dropout": 0.5
}

class Params:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

params = Params(**params)

### Regression Model

In [181]:
class BertRegressor(nn.Module):
  def __init__(self, dropout=params.dropout):
    super(BertRegressor, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, 1)

  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)

    return linear_output

### Dataset Class

In [182]:
class ChessDataset(torch.utils.data.Dataset):
  def __init__(self, moves, labels):
      tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
      self.labels = torch.Tensor(labels)
      self.moves = [tokenizer(move, 
                              padding='max_length', max_length = 256, truncation=True,
                              return_tensors="pt") for move in moves]

  def __len__(self):
      return len(self.labels)

  def __getitem__(self, idx):
      return self.moves[idx], self.labels[idx]

Helper Functions

In [183]:
def rating_to_bucket(rating):
  if rating < 1300:
    return 0
  elif rating >= 1300 and rating < 1500:
    return 1
  elif rating >= 1500 and rating < 1700:
    return 2  
  elif rating >= 1700 and rating < 1900:
    return 3
  elif rating >= 1900 and rating < 2100:
    return 4
  elif rating >= 2100:
    return 5

In [234]:
def get_accuracy(model_output, label):
  output = model_output.cpu().detach().numpy()
  label = label.cpu().detach().numpy()
  
  pred_rating = torch.Tensor(scaler.inverse_transform(output))
  real_rating = torch.Tensor(scaler.inverse_transform(label))

  print(f"Model Output: {output}")
  print(f"Prediction: {pred_rating}, Truth: {real_rating}")
  print(f"L1 Loss: {criterion(pred_rating, real_rating)}")
  pred_bucket = np.array(list(map(rating_to_bucket, pred_rating)))
  real_bucket = np.array(list(map(rating_to_bucket, real_rating)))

  print(f"pred bucket: {pred_bucket} // real_bucket: {real_bucket}")

  acc = (pred_bucket == real_bucket).sum().item()
  return acc

### Training Function

In [236]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer):
    num_batches = len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = params.warmup_steps, num_training_steps = num_batches * params.epochs)
    print(f"Number of batches: {num_batches}")
    for epoch_num in range(params.epochs):
        total_acc_train = 0
        total_loss_train = 0

        for batch_idx, (train_input, train_label) in enumerate(train_dataloader):
          train_label = train_label.to(device)
          mask = train_input['attention_mask'].to(device)
          input_id = train_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)
          
          batch_loss = criterion(output.squeeze(), train_label.squeeze())
          total_loss_train += batch_loss.item()
          
          acc = get_accuracy(output, train_label)
          total_acc_train += acc

          batch_loss.backward()

          # Gradient Accumulation
          if ((batch_idx + 1) % params.accum_iter == 0) or (batch_idx + 1 == num_batches):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

          if batch_idx % 50 == 0:
            print(f"Batch Number: {batch_idx}")
            print(f"Model Output: {output}")
            print(f"Train Labels: {train_label.tolist()}")
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

          for val_input, val_label in val_dataloader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()
            
            acc = (output == val_label).sum().item()
            total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / (params.batch_size * len(train_dataloader)): .3f} \
            | Train Accuracy: {total_acc_train / (params.batch_size * len(train_dataloader)): .3f} \
            | Val Loss: {total_loss_val / (params.batch_size * len(val_dataloader)): .3f} \
            | Val Accuracy: {total_acc_val / (params.batch_size * len(val_dataloader)): .3f}')
        
        torch.save(model.state_dict(), model_path)

### Test Function

In [231]:
def test(model, test_dataloader, criterion, scaler):
  total_acc_test = 0
  with torch.no_grad():
    for test_input, test_label in test_dataloader:
      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)
      
      acc = get_accuracy(output, test_label)
      total_acc_test += acc
       
    print(f"Test Accuracy: {total_acc_test / len(test_dataloader): .3f}")

### Loading Data

In [186]:
%%capture
chess_data = pd.read_csv(csv_path)[0:3000]
z = chess_data["Bucket"]
chess_data = chess_data[["Moves", "Elo"]]

balancer = RandomUnderSampler()
chess_data, z = balancer.fit_resample(chess_data, z)
rating_counts = z.value_counts().sort_index()

X = chess_data["Moves"]
y = chess_data["Elo"]

scaler = StandardScaler()
y = scaler.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, shuffle=True) # Splits into train/test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle=True) # Splits train into train/val

train_data, val_data, test_data = ChessDataset(X_train, y_train), ChessDataset(X_val, y_val), ChessDataset(X_test, y_test)

In [187]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=params.batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=params.batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data)

### Initialize Model

In [203]:
%%capture
model = BertRegressor()
model = model.to(device)
criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr = params.learning_rate)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Run Training

In [216]:
train(model, train_dataloader, val_dataloader, criterion, optimizer)

Number of batches: 698
Batch Number: 0
Model Output: tensor([[0.4213]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train Labels: [[-0.6240825057029724]]
Batch Number: 50
Model Output: tensor([[-0.0037]], device='cuda:0', grad_fn=<AddmmBackward0>)
Train Labels: [[-0.16755814850330353]]


KeyboardInterrupt: ignored

### Run Testing

In [235]:
test_model_name = "06_01_2023_20_27_regression_model.pth"
test_model_path = f"{folder}/{test_model_name}"

test_model = BertRegressor()
test_model = test_model.to(device)
test_model.load_state_dict(torch.load(test_model_path))
test(test_model, test_dataloader, criterion, scaler)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Output: [[0.26046306]]
Prediction: tensor([[1793.1976]]), Truth: tensor([[1956.]])
L1 Loss: 26504.611328125
pred bucket: [3] // real_bucket: [4]
Model Output: [[0.01168938]]
Prediction: tensor([[1707.6437]]), Truth: tensor([[1566.]])
L1 Loss: 20062.931640625
pred bucket: [3] // real_bucket: [2]
Model Output: [[-0.07774206]]
Prediction: tensor([[1676.8881]]), Truth: tensor([[1548.]])
L1 Loss: 16612.1328125
pred bucket: [2] // real_bucket: [2]
Model Output: [[-0.30640772]]
Prediction: tensor([[1598.2493]]), Truth: tensor([[2007.]])
L1 Loss: 167077.15625
pred bucket: [2] // real_bucket: [4]
Model Output: [[0.39576107]]
Prediction: tensor([[1839.7271]]), Truth: tensor([[2032.]])
L1 Loss: 36968.88671875
pred bucket: [3] // real_bucket: [4]
Model Output: [[-0.20407942]]
Prediction: tensor([[1633.4402]]), Truth: tensor([[1272.]])
L1 Loss: 130639.0078125
pred bucket: [2] // real_bucket: [0]
Model Output: [[-0.1638879]]
Prediction: tensor([[1647.2622]]), Truth: tensor([[1619.]])
L1 Loss: 